In [5]:
import pandas as pd
import random
from fpdf import FPDF

DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
PERIODS_PER_DAY = 7
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
GENERATIONS = 500
random.seed(42)

def load_excel_data(file):
    return pd.read_excel(file).applymap(lambda x: x.strip() if isinstance(x, str) else x)

def preprocess_course_data(course_data):
    subject_data = []
    class_subject_staff = {}
    for _, row in course_data.iterrows():
        sections = str(row["Class"]).split(",")
        staff_list = str(row["Staff Name"]).split(",")
        for i, section in enumerate(sections):
            subject_info = {
                "Course Name": row["Course Name"].strip(),
                "Course Short Name": row["Course Short Name"].strip(),
                "Class": section.strip(),
                "Room Type": row["Room type"].strip(),
                "Staff Name": staff_list[i].strip() if i < len(staff_list) else staff_list[0].strip(),
                "Max Hours": int(row["Maximum Duration  per Week"])
            }
            subject_data.append(subject_info)
            class_subject_staff.setdefault(section.strip(), []).append(subject_info)
    return subject_data, class_subject_staff

def process_data(course_data, students_data, room_data):
    subjects, class_subject_staff = preprocess_course_data(course_data)
    mca_classes = students_data[students_data["Course"] == "MCA"]["Class"].dropna().unique().tolist()
    theory_rooms = room_data[room_data["Room type"] == "Lecture Hall"]["Room Name "].dropna().tolist()
    lab_rooms = room_data[room_data["Room type"] == "Laboratory"]["Room Name "].dropna().tolist()
    return subjects, class_subject_staff, mca_classes, theory_rooms, lab_rooms

def assign_room(room_schedule, room_list, day, period, span=1):
    for room in room_list:
        if all(room_schedule.get((room, day, period + i)) is None for i in range(span)):
            return room
    return None

def is_room_free(room_schedule, room, day, period, span=1):
    return all(room_schedule.get((room, day, period + i)) is None for i in range(span))

def repair_timetable(timetable, subjects, theory_rooms, lab_rooms):
    room_schedule = {}
    staff_schedule = {}
    for cls, daily in timetable.items():
        available_subjects = [s for s in subjects if s["Class"] == cls]
        subject_count = {s["Course Name"]: 0 for s in available_subjects}
        for day in DAYS:
            used_today = {}
            prev_sub = None
            i = 0
            while i < PERIODS_PER_DAY:
                if daily[day][i] is None:
                    valid_subjects = [
                        s for s in available_subjects
                        if subject_count[s["Course Name"]] < s["Max Hours"]
                        and used_today.get(s["Course Name"], 0) < 2
                        and s["Course Name"] != prev_sub
                    ]
                    random.shuffle(valid_subjects)
                    for subject in valid_subjects:
                        room_list = lab_rooms if subject["Room Type"] == "Laboratory" else theory_rooms
                        span = 2 if subject["Room Type"] == "Laboratory" else 1
                        if i + span > PERIODS_PER_DAY:
                            continue
                        room = assign_room(room_schedule, room_list, day, i, span)
                        if room and is_room_free(room_schedule, room, day, i, span):
                            staff = subject["Staff Name"]
                            if all((staff, day, i + k) not in staff_schedule for k in range(span)):
                                for k in range(span):
                                    daily[day][i + k] = (subject["Course Short Name"], room, staff)
                                    room_schedule[(room, day, i + k)] = cls
                                    staff_schedule[(staff, day, i + k)] = cls
                                subject_count[subject["Course Name"]] += span
                                used_today[subject["Course Name"]] = used_today.get(subject["Course Name"], 0) + span
                                prev_sub = subject["Course Name"]
                                i += span
                                break
                    else:
                        i += 1
                else:
                    prev_sub = daily[day][i][0]
                    i += 1
    return timetable

def generate_initial_population(mca_classes, subjects, theory_rooms, lab_rooms):
    population = []

    for _ in range(POPULATION_SIZE):
        timetable = {cls: {day: [None] * PERIODS_PER_DAY for day in DAYS} for cls in mca_classes}
        room_schedule = {}
        staff_schedule = {}

        for cls in mca_classes:
            class_subjects = [s for s in subjects if s["Class"] == cls]
            subject_count = {s["Course Name"]: 0 for s in class_subjects}

            for day in DAYS:
                used_today = {}
                prev_sub = None
                period = 0

                while period < PERIODS_PER_DAY:
                    # Get valid subjects for this slot
                    valid_subjects = [
                        s for s in class_subjects
                        if subject_count[s["Course Name"]] < s["Max Hours"]
                        and used_today.get(s["Course Name"], 0) < 2
                        and s["Course Name"] != prev_sub
                    ]

                    random.shuffle(valid_subjects)
                    assigned = False

                    for subject in valid_subjects:
                        room_list = lab_rooms if subject["Room Type"] == "Laboratory" else theory_rooms
                        span = 2 if subject["Room Type"] == "Laboratory" else 1
                        if period + span > PERIODS_PER_DAY:
                            continue

                        # Find an available room
                        for room in room_list:
                            if all(room_schedule.get((room, day, period + i)) is None for i in range(span)):
                                # Check staff availability
                                staff = subject["Staff Name"]
                                if all((staff, day, period + i) not in staff_schedule for i in range(span)):
                                    # Assign subject
                                    for i in range(span):
                                        timetable[cls][day][period + i] = (subject["Course Short Name"], room, staff)
                                        room_schedule[(room, day, period + i)] = cls
                                        staff_schedule[(staff, day, period + i)] = cls
                                    subject_count[subject["Course Name"]] += span
                                    used_today[subject["Course Name"]] = used_today.get(subject["Course Name"], 0) + span
                                    prev_sub = subject["Course Name"]
                                    period += span
                                    assigned = True
                                    break
                        if assigned:
                            break

                    if not assigned:
                        period += 1  # Try next period

        population.append(timetable)

    return population


def fitness(timetable, subjects):
    score = 0
    room_usage = {}
    subject_weekly = {}
    max_hours_lookup = {(s["Class"], s["Course Short Name"]): s["Max Hours"] for s in subjects}

    for cls in timetable:
        for day in DAYS:
            used_today = {}
            prev_sub = None
            for i, entry in enumerate(timetable[cls][day]):
                if entry:
                    sub, room, _ = entry
                    if (room, day, i) in room_usage:
                        score -= 50
                    else:
                        room_usage[(room, day, i)] = cls
                    used_today[sub] = used_today.get(sub, 0) + 1
                    if used_today[sub] > 2:
                        score -= 20
                    if sub == prev_sub:
                        score -= 15
                    subject_weekly[(cls, sub)] = subject_weekly.get((cls, sub), 0) + 1
                    prev_sub = sub
    for (cls, sub), count in subject_weekly.items():
        allowed = max_hours_lookup.get((cls, sub), 4)
        if count > allowed:
            score -= 30
    score += 10000 - sum(1 for cls in timetable for d in DAYS for p in timetable[cls][d] if p is None)
    return score

def crossover(parent1, parent2):
    child = {}
    for cls in parent1:
        child[cls] = {}
        for day in DAYS:
            child[cls][day] = parent1[cls][day] if random.random() > 0.5 else parent2[cls][day]
    return child

def mutate(timetable):
    class_name = random.choice(list(timetable.keys()))
    day = random.choice(DAYS)
    p1, p2 = random.sample(range(PERIODS_PER_DAY), 2)
    timetable[class_name][day][p1], timetable[class_name][day][p2] = timetable[class_name][day][p2], timetable[class_name][day][p1]
    return timetable

def genetic_algorithm(subjects, class_subject_staff, mca_classes, theory_rooms, lab_rooms):
    population = generate_initial_population(mca_classes, subjects, theory_rooms, lab_rooms)
    for _ in range(GENERATIONS):
        population.sort(key=lambda t: fitness(t, subjects), reverse=True)
        next_gen = population[:10]
        while len(next_gen) < POPULATION_SIZE:
            p1, p2 = random.sample(population[:20], 2)
            child = crossover(p1, p2)
            if random.random() < MUTATION_RATE:
                child = mutate(child)
            repaired = repair_timetable(child, subjects, theory_rooms, lab_rooms)
            next_gen.append(repaired)
        population = next_gen
    return max(population, key=lambda t: fitness(t, subjects))


def export_to_pdf(timetable, class_subject_staff, filename="Generated_Timetable.pdf"):
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.set_auto_page_break(auto=True, margin=10)
    pdf.set_font("Arial", size=10)
    
    for class_name, schedule in timetable.items():
        pdf.add_page()
        pdf.set_font("Arial", "B", 14)
        pdf.cell(270, 10, f"Timetable for {class_name}", ln=True, align="C")
        pdf.ln(5)
        
        pdf.set_font("Arial", size=10)
        pdf.cell(40, 10, "Day", border=1, align="C")
        for period in range(1, PERIODS_PER_DAY + 1):
            pdf.cell(35, 10, f"P{period}", border=1, align="C")
        pdf.ln()
        
        for day in DAYS:
            pdf.cell(40, 10, day, border=1, align="C")
            for period in range(PERIODS_PER_DAY):
                entry = schedule[day][period]
                if entry:
                    subject, room, _ = entry
                    if subject.strip().lower() == "library":
                        pdf.cell(35, 10, f"{subject}", border=1, align="C")
                    else:
                        pdf.cell(35, 10, f"{subject}\n({room})", border=1, align="C")
                else:
                    pdf.cell(35, 10, "-", border=1, align="C")
            pdf.ln()

        pdf.ln(5)
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Staff and Course Assignments", ln=True, align="L")
        pdf.set_font("Arial", size=12)
        if class_name in class_subject_staff:
            for entry in class_subject_staff[class_name]:
                pdf.cell(0, 10, f"{entry['Staff Name']} - {entry['Course Name']} ({entry['Course Short Name']})", ln=True, align="L")
    
    pdf.output(filename)
    return filename
